In [ ]:
from ROOT import TFile,TTree,TCanvas,TH1F, gStyle, TLatex, gPad, TLegend, TLorentzVector, TH2F, TLine, TF1, TBox, RDataFrame, TGraphErrors, TFitResultPtr, TF2, TChain
import ROOT
import numpy as np
from math import log10, floor

In [37]:
def round_sig(x, sig=2):
    if(x==0):
        return x
    return round(x, sig-int(floor(log10(abs(x))))-1)

def round_to_reference(x, y):
    if(y==0):
        return x
    return round(x, -int(floor(log10(y))))

def val_and_err(val,err,sig=2):
    err = round_sig(err,sig)
    val = round_to_reference(val,err)
    return val,err

In [ ]:
def gethists_A_LU(d,bintype,bins,phihbins,phiRbins,signal_range):
    nPlots = len(bins-1)
    hists_sig_plus = []
    hists_sig_minus = []
    
    for i in range(nPlots-1):
        final_cut = " {} > {:.5f} && {} < {:.5f}".format(bintype,bins[i],bintype,bins[i+1])
        hists_sig_plus.append(d.Filter(final_cut + " && helicity == 1 && Mdiphoton > {} && Mdiphoton < {}".format(signal_range[0],signal_range[1]))
                      .Histo2D(("h1","h1",phihbins,-3.141592,3.141592,phiRbins,-3.141592,3.141592),"phi_h","phi_R","Signal"))

        hists_sig_minus.append(d.Filter(final_cut + " && helicity == -1 && Mdiphoton > {} && Mdiphoton < {}".format(signal_range[0],signal_range[1]))
                      .Histo2D(("h2","h2",phihbins,-3.141592,3.141592,phiRbins,-3.141592,3.141592),"phi_h","phi_R","Signal"))
    
    
    return hists_sig_plus, hists_sig_minus

In [ ]:
def process_A_LU(hists,pol,polerr):
    
    hists_sig_plus = hists[0]
    hists_sig_minus = hists[1]
    
    f_=TF2("f_","[0]*sin(2*x-2*y)+[1]*sin(x-y)+[2]*sin(-x+2*y)+[3]*sin(y)+[4]*sin(x)+[5]*sin(2*x-y)+[6]*sin(3*x-2*y)",-3.141592,3.141592,-3.141592,3.141592)
    
    
    fitparams = np.zeros((len(hists[0]),7))
    fiterrors = np.zeros((len(hists[0]),7))
    
    for i in range(len(hists[0])):
        hsp1 = hists_sig_plus[i].GetValue().Clone()
        hsp2 = hists_sig_plus[i].GetValue().Clone()
        hsm = hists_sig_minus[i].GetValue().Clone()
        
        # Set Sumw2 for proper error propagation when dividing
        hsp1.Sumw2()
        hsp2.Sumw2()
        hsm.Sumw2()
        
        # For the sig+bg range (N+ - N-)/(N+ + N-)
        hsp1.Add(hsm,-1)
        hsp2.Add(hsm,+1)
        hsp1.Divide(hsp2)
               # Divide by weighted polarization
        h_sig = hsp1.Clone()
        h_sig.Scale(1/pol)
        h_sig.Fit(f_,"SR Q0")
        fitparams[i] = np.array([f_.GetParameter(i) for i in range(7)])
        fiterrors[i] = np.array([f_.GetParError(i) for i in range(7)])

    return fitparams, fiterrors

In [ ]:
mods = ["sin(2#phi_{h}-2#phi_{R})","sin(#phi_{h}-#phi_{R})", "sin(-#phi_{h}+2#phi_{R})" , "sin(#phi_{R})", "sin(#phi_{h})", "sin(2#phi_{h}-#phi{R})" , "sin(3#phi_{h}-2#phi_{R})"]

In [ ]:
def get_asym_graph(bins,fitpars,errors,color,marker,plotyranges,idx_mods,plotstr):
    tges=[]

    for i in range(len(idx_mods)):

        idx = idx_mods[i]
        modulation_name = mods[idx]

        # Create TGraphErrors for plotting
        tges.append(TGraphErrors(len(bins)-1))
        tges[i].SetMarkerStyle(marker)
        tges[i].SetMarkerColor(color)
        tges[i].SetLineColor(color)
        tges[i].SetTitle("{}".format(modulation_name)+";{};".format(plotstr))

        for j in range(tges[i].GetN()):
            x = np.mean(bins[j:j+2])
            y = fitpars[j,idx]
            yerr = errors[j,idx]
            tges[i].SetPoint(j,x,y)
            tges[i].SetPointError(j,0,yerr)
            tges[i].GetYaxis().SetRangeUser(plotyranges[i][0],plotyranges[i][1])
    return tges